# Introduction to the Node Normalization Service (NodeNorm)

## Introduction

The [Node Normalization Service](https://nodenormalization-sri.renci.org/docs) (NodeNorm) takes an input [CURIE](https://en.wikipedia.org/wiki/CURIE), and returns:

* The preferred CURIE for this entity
* All other known equivalent identifiers for the entity
* Semantic types for the entity as defined by the [Biolink Model](https://biolink.github.io/biolink-model/)

The data currently served by the Node Normalization Service is created by [Babel](https://github.com/TranslatorSRI/Babel), a pipeline for combining identifiers from hundreds of data sources including ontologies and publicly accessible databases into a series of _cliques_ -- sets of identifiers ordered as per the preferred prefix list for that semantic type.

## Instances of NodeNorm

For the examples in this document, we will use the _development_ version of the Node Normalization Service, hosted at https://nodenormalization-sri.renci.org/ by the Renaissance Computing Institute (RENCI) at the University of North Carolina. This version is updated more frequently than the production instance listed below.

The production instance of NodeNorm is hosted by the NCATS Translator project at https://nodenorm.transltr.io/docs, and may be older than the development version hosted below. As with other NCATS Translator tools, a [CI instance](https://nodenorm.ci.transltr.io/docs) and a [Test instance](https://nodenorm.test.transltr.io/docs) have also been deployed, but are not likely to be useful for non-Translator users.

## Metadata

There are two metadata services that can be used to find out what sorts of results have been incorporated into the Node Normalization service.  These return the semantic types that are included, and the prefixes included for each type.

#### Which types have been normalized?

In [1]:
import json 
import requests

result = requests.get('https://nodenormalization-sri.renci.org/get_semantic_types')
print( json.dumps( result.json(), indent = 2))

{
  "semantic_types": {
    "types": [
      "biolink:Protein",
      "biolink:Pathway",
      "biolink:Behavior",
      "biolink:BiologicalProcess",
      "biolink:GeneGroupingMixin",
      "biolink:AdministrativeEntity",
      "biolink:Attribute",
      "biolink:MolecularEntity",
      "biolink:Entity",
      "biolink:GeneFamily",
      "biolink:BiologicalEntity",
      "biolink:DiseaseOrPhenotypicFeature",
      "biolink:MolecularActivity",
      "biolink:OrganismTaxon",
      "biolink:MolecularMixture",
      "biolink:GenomicEntity",
      "biolink:GeographicLocation",
      "biolink:ActivityAndBehavior",
      "biolink:MacromolecularComplex",
      "biolink:GeneProductMixin",
      "biolink:PhysicalEssenceOrOccurrent",
      "biolink:StudyPopulation",
      "biolink:MacromolecularMachineMixin",
      "biolink:InformationContentEntity",
      "biolink:SubjectOfInvestigation",
      "biolink:Device",
      "biolink:OrganismalEntity",
      "biolink:Polypeptide",
      "biolink:Named

#### Which prefixes are supported?

Even if a semantic type has some identifier equivalence, not every vocabulary has been included.  To see which vocabularies are likely to give useful results for a specific Biolink class, call:

In [3]:
result = requests.get('https://nodenormalization-sri.renci.org/get_curie_prefixes/',
                     params={'semantic_type':"biolink:ChemicalEntity"})
print( json.dumps( result.json(), indent = 2))

{
  "biolink:ChemicalEntity": {
    "curie_prefix": {
      "PUBCHEM.COMPOUND": "124460882",
      "INCHIKEY": "110336273",
      "CHEMBL.COMPOUND": "2399641",
      "CHEBI": "196937",
      "CAS": "3556698",
      "HMDB": "217920",
      "UNII": "126327",
      "MESH": "258102",
      "KEGG.COMPOUND": "16020",
      "UMLS": "549120",
      "DRUGBANK": "15179",
      "DrugCentral": "4995",
      "GTOPDB": "12347",
      "RXCUI": "123395"
    }
  }
}


More than one type can be queried:

In [4]:
result = requests.get('https://nodenormalization-sri.renci.org/get_curie_prefixes/',
                     params={'semantic_type':["biolink:ChemicalEntity","biolink:Disease"]})
print( json.dumps( result.json(), indent = 2))

{
  "biolink:ChemicalEntity": {
    "curie_prefix": {
      "PUBCHEM.COMPOUND": "124460882",
      "INCHIKEY": "110336273",
      "CHEMBL.COMPOUND": "2399641",
      "CHEBI": "196937",
      "CAS": "3556698",
      "HMDB": "217920",
      "UNII": "126327",
      "MESH": "258102",
      "KEGG.COMPOUND": "16020",
      "UMLS": "549120",
      "DRUGBANK": "15179",
      "DrugCentral": "4995",
      "GTOPDB": "12347",
      "RXCUI": "123395"
    }
  },
  "biolink:Disease": {
    "curie_prefix": {
      "UMLS": "336767",
      "SNOMEDCT": "94514",
      "MEDDRA": "46033",
      "MONDO": "24012",
      "DOID": "11498",
      "NCIT": "25540",
      "HP": "2182",
      "EFO": "3711",
      "orphanet": "10780",
      "MESH": "11238",
      "OMIM": "10314",
      "ICD10": "2429",
      "ICD9": "2218",
      "OMIM.PS": "556",
      "KEGG.DISEASE": "40",
      "MP": "1"
    }
  }
}


Or you can just ask for all the prefix counts for every type:

In [8]:
result = requests.get('https://nodenormalization-sri.renci.org/get_curie_prefixes')
responses = result.json()

# Display only the first three responses. 
print( json.dumps( dict(list(responses.items())[:3]), indent = 2))

{
  "biolink:Pathway": {
    "curie_prefix": {
      "REACT": "21784",
      "SMPDB": "30248",
      "GO": "642",
      "PANTHER.PATHWAY": "175"
    }
  },
  "biolink:OrganismTaxon": {
    "curie_prefix": {
      "UMLS": "749649",
      "NCBITaxon": "2546095",
      "MESH": "70628"
    }
  },
  "biolink:GeneFamily": {
    "curie_prefix": {
      "PANTHER.FAMILY": "26138",
      "HGNC.FAMILY": "1807"
    }
  }
}


## Normalization

Given one or more Compact URIs (CURIES), `get_normalized_node` will return a list of equivalent identifiers for the entity, along with the Translator-preferred identifier, and the semantic type(s) for the entity.  This service is merely returning pre-computed values, and does no equivalence inference on its own.  If a CURIE is unknown to it, then null is returned.

In this example, `get_normalized_node` is called with a MeSH identifier.   MeSH contains many different semantic types, but the service correctly identifies the term.

In [4]:
result = requests.get('https://nodenormalization-sri.renci.org/get_normalized_nodes',
                     params={'curie':"MESH:D014867"})
print( json.dumps( result.json(), indent = 2))

{
  "MESH:D014867": {
    "id": {
      "identifier": "PUBCHEM.COMPOUND:22247451",
      "label": "Hydron;hydroxide"
    },
    "equivalent_identifiers": [
      {
        "identifier": "PUBCHEM.COMPOUND:22247451",
        "label": "Hydron;hydroxide"
      },
      {
        "identifier": "PUBCHEM.COMPOUND:962",
        "label": "Water"
      },
      {
        "identifier": "CHEMBL.COMPOUND:CHEMBL1098659",
        "label": "WATER"
      },
      {
        "identifier": "UNII:059QF0KO0R"
      },
      {
        "identifier": "CHEBI:15377",
        "label": "water"
      },
      {
        "identifier": "DRUGBANK:DB09145"
      },
      {
        "identifier": "MESH:D014867",
        "label": "Water"
      },
      {
        "identifier": "HMDB:HMDB0002111"
      },
      {
        "identifier": "KEGG.COMPOUND:C00001",
        "label": "H2O"
      },
      {
        "identifier": "INCHIKEY:IKBQPNVYXHKVJS-LVZFUZTISA-N"
      }
    ],
    "type": [
      "biolink:ChemicalSubstance",
    

To improve performance, multiple CURIEs may be batched into a single function call as a POST:

In [5]:
result = requests.post('https://nodenormalization-sri.renci.org/get_normalized_nodes',
                     json={"curies":["HP:0007354", "HGNC:613", "CURIE:NOTHING"]})
print( json.dumps( result.json(), indent = 2))

{
  "HP:0007354": {
    "id": {
      "identifier": "MONDO:0004976",
      "label": "amyotrophic lateral sclerosis"
    },
    "equivalent_identifiers": [
      {
        "identifier": "MONDO:0004976",
        "label": "amyotrophic lateral sclerosis"
      },
      {
        "identifier": "DOID:332"
      },
      {
        "identifier": "OMIM:MTHU030638"
      },
      {
        "identifier": "OMIM:MTHU038375"
      },
      {
        "identifier": "ORPHANET:803"
      },
      {
        "identifier": "EFO:0000253",
        "label": "amyotrophic lateral sclerosis"
      },
      {
        "identifier": "UMLS:C0002736",
        "label": "Amyotrophic Lateral Sclerosis"
      },
      {
        "identifier": "UMLS:C0393554",
        "label": "Amyotrophic Lateral Sclerosis With Dementia"
      },
      {
        "identifier": "MESH:D000690",
        "label": "Amyotrophic Lateral Sclerosis"
      },
      {
        "identifier": "MEDDRA:10002026"
      },
      {
        "identifier": "NCI

## TRAPI

Node normalization will now operate on TRAPI messages (version 1.0)

Here we have a message in terms of HGNC and DOID, and the normalizer returns a message using NCBIGene and MONDO.

In [11]:
trapi_message = {
    "message": {
        "query_graph": {
            "nodes": {
                "n1": {
                    "id": "HGNC:11603",
                    "category": [
                        "biolink:Gene"
                    ]
                },
                "n2": {
                    "category": [
                        "biolink:Disease"
                    ]
                }
            },
            "edges": {
                "e1": {
                    "subject": "n1",
                    "object": "n2"
                }
            }
        },
        "knowledge_graph": {
            "nodes": {
                "HGNC:11603": {
                    "name": "TBX4",
                    "category": [
                        "biolink:Gene"
                    ]
                },
                "DOID:3083": {
                    "name": "chronic obstructive pulmonary disease",
                    "category": [
                        "biolink:Disease"
                    ]
                }
            },
            "edges": {
                "2d38345a-e9bf-4943-accb-dccba351dd04": {
                    "subject": "NCBIGene:9496",
                    "object": "DOID:3083",
                    "predicate": "biolink:related_to",
                    "relation": "RO:0003304"
                }
            }
        },
        "results": [
            {
                "node_bindings": {
                    "n1": [
                        {
                            "id": "HGNC:11603"
                        }
                    ],
                    "n2": [
                        {
                            "id": "DOID:3083"
                        }
                    ]
                },
                "edge_bindings": {
                    "e1": [
                        {
                            "id": "2d38345a-e9bf-4943-accb-dccba351dd04"
                        }
                    ]
                }
            }
        ]
    }
}

In [12]:
result = requests.post('https://nodenormalization-sri.renci.org/response',json=trapi_message)
print(result.status_code)

200


In [13]:
print(json.dumps(result.json(), indent=2))

{
  "message": {
    "query_graph": {
      "nodes": {
        "n1": {
          "id": "NCBIGene:9496",
          "category": [
            "biolink:Gene"
          ],
          "is_set": false
        },
        "n2": {
          "id": null,
          "category": [
            "biolink:Disease"
          ],
          "is_set": false
        }
      },
      "edges": {
        "e1": {
          "subject": "n1",
          "object": "n2",
          "predicate": null,
          "relation": null
        }
      }
    },
    "knowledge_graph": {
      "nodes": {
        "NCBIGene:9496": {
          "category": [
            "biolink:Gene",
            "biolink:Gene",
            "biolink:GenomicEntity",
            "biolink:MolecularEntity",
            "biolink:BiologicalEntity",
            "biolink:NamedThing",
            "biolink:Entity"
          ],
          "name": "TBX4",
          "attributes": [
            {
              "type": "biolink:same_as",
              "value": [
     